In [2]:
#! -*- coding:utf-8 -*-
# 情感分析例子

import os, json
import numpy as np
import pandas as pd
from bert4keras.backend import keras, set_gelu
from bert4keras.tokenizers import Tokenizer
from bert4keras.models import build_transformer_model
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open
from keras.layers import Lambda, Dense
import jieba
jieba.initialize()

num_classes = 2
maxlen = 128
batch_size = 32

# bert配置
config_path = 'chinese_wobert_plus_L-12_H-768_A-12/bert_config.json'
checkpoint_path = 'chinese_wobert_plus_L-12_H-768_A-12/bert_model.ckpt'
dict_path = 'chinese_wobert_plus_L-12_H-768_A-12/vocab.txt'

def load_data(filename):
    """加载数据
    单条格式：(文本, 标签id)
    """
    data = pd.read_excel(filename)
    
    D = []
    for index,row in data.iterrows():
        D.append((row['question_des'], int(row['label'])))
    return D

# 加载数据集
train_data = load_data('traindata_new.xlsx')
valid_data = load_data('validdata_new.xlsx')

print(len(train_data))
print(len(valid_data))

# 建立分词器
tokenizer = Tokenizer(
    dict_path,
    do_lower_case=True,
    pre_tokenize=lambda s: jieba.cut(s, HMM=False)
)


class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        for is_end, (text, label) in self.sample(random):
            token_ids, segment_ids = tokenizer.encode(text, maxlen=maxlen)
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append([label])
            if len(batch_token_ids) == self.batch_size or is_end:
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []


bert = build_transformer_model(
    config_path,
    checkpoint_path,
    return_keras_model=False,
)

output = bert.model.output
output = Lambda(lambda x: x[:, 0], name='CLS-token')(output)
output = Dense(
    units=num_classes,
    activation='softmax',
    kernel_initializer=bert.initializer
)(output)

model = keras.models.Model(bert.model.input, output)
model.summary()

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=Adam(1e-5),  # 用足够小的学习率
    metrics=['accuracy'],
)

# 转换数据集
train_generator = data_generator(train_data, batch_size)
valid_generator = data_generator(valid_data, batch_size)
# test_generator = data_generator(test_data, batch_size)


def evaluate(data):
    total, right = 0., 0.
    for x_true, y_true in data:
        y_pred = model.predict(x_true).argmax(axis=1)
        y_true = y_true[:, 0]
        total += len(y_true)
        right += (y_true == y_pred).sum()
    return right / total


class Evaluator(keras.callbacks.Callback):
    def __init__(self):
        self.best_val_acc = 0.

    def on_epoch_end(self, epoch, logs=None):
        val_acc = evaluate(valid_generator)
        if val_acc > self.best_val_acc:
            self.best_val_acc = val_acc
            model.save_weights('best_model_sentiment_plus.weights')
        print(
            u'val_acc: %.5f, best_val_acc: %.5f\n' %
            (val_acc, self.best_val_acc)
        )

if False:

    evaluator = Evaluator()

    model.fit_generator(
        train_generator.forfit(),
        steps_per_epoch=len(train_generator),
        epochs=10,
        callbacks=[evaluator]
    )

    model.load_weights('best_model_sentiment_plus.weights')
#     print(u'final test acc: %05f\n' % (evaluate(test_generator)))

else:

    model.load_weights('best_model_sentiment_plus.weights')


Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.916 seconds.
Prefix dict has been built succesfully.


2816
705




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.






Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    38400000    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536

In [ ]:
best_val_acc: 0.97447

In [15]:
# pred_data = pd.read_csv("售后_fengwei31_敏感词数据提取_20220901153952.csv",encoding='gb18030',engine='python')
pred_data = pd.read_excel('validdata_new.xlsx')
data_list = list(set(list(pred_data.question_des)))
test_data = []
for i in data_list:
    test_data.append((i,0))

test_generator = data_generator(test_data, 64)

from tqdm import tqdm
y_pred_list = []
for x_true, y_true in tqdm(test_generator):
    y_pred = model.predict(x_true).argmax(axis=1)
    y_pred_list+=list(y_pred)

des_label_dict = {}
for i in range(len(data_list)):
    des_label_dict[data_list[i]] = y_pred_list[i]

def func(x):
    if x == 0:
        return '是'
    else:
        return '否'
    
pred_data['预测'] = pred_data['question_des'].map(des_label_dict)
    
    
pred_data['机器预测_是否需要转人工'] = pred_data['预测'].map(func)

pred_data['人工标注_是否需要转人工'] = pred_data['label'].map(func)

pred_data['服务单号'] = pred_data['afs_id']
pred_data['问题描述'] = pred_data['question_des']

last_data = pred_data[['服务单号','问题描述','人工标注_是否需要转人工','机器预测_是否需要转人工']]
last_data.to_excel("验证数据20220928_fengwei.xlsx")

100%|██████████| 8/8 [00:02<00:00,  3.22it/s]
